In [ ]:
%%capture
try:
  already_installed
except:  
    !pip install kora
    !pip install dgl
    !pip install dgl-cu101
    !pip install --pre dgl-cu101
    !pip install dgllife
    # !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
    # !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
    # !pip install -q torch-geometric
    !pip install torch_optimizer
    #!pip install pytorch_lightning
    import sys
    import time
    import dgl
    import dgllife
    import torch 
    import pickle
    from IPython.display import clear_output
    import networkx as nx
    import dgl.function as fn
    import torch.nn.functional as F
    import torch.nn as nn
    sys.path.append('/usr/local/lib/python3.6/dist-packages/') # For using RDKit utilizing KORA
    import kora.install.rdkit
    from rdkit import Chem
    from rdkit.Chem import Descriptors
    from rdkit.Chem.Draw import IPythonConsole
    from rdkit.Chem import ChemicalFeatures
    from rdkit import RDConfig
    from dgllife.utils import featurizers as fs
    from torch.utils.data import DataLoader
    import networkx as nx
    import matplotlib.pyplot as plt
    import torch_optimizer as optim
    # import pytorch_lightning as pl  
    # from pytorch_lightning.metrics.classification import AUROC
    # from pytorch_lightning.metrics.classification import AveragePrecision
    # from pytorch_lightning.metrics.classification import Recall
    # from pytorch_lightning.metrics.classification import Precision
    # from pytorch_lightning.metrics.classification import ConfusionMatrix
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import numpy as np
    import itertools
    import os
    %matplotlib inline
    import warnings
    warnings.filterwarnings("ignore")
    # Fix Seeds
    # Set a seed value
    seed_value= 213 
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random 
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)
    # 4. Set `pytorch` pseudo-random generator at a fixed value
    torch.manual_seed(seed_value)
    # 5. Set `dgl` pseudo-random generator at a fixed value
    #dgl.seed(seed_value) # dose not work on CPU recently 
    # 6. For layers that introduce randomness like dropout, make sure to set seed values 
    # model.add(Dropout(0.25, seed=seed_value)) 
    already_installed=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Node and Edge Featurizer

def atom_is_in_ring_list_one_hot(atom, allowable_set=None, encode_unknown=False):
          list = [3,4,5] # List denotes the size of cycles
          return [atom.IsInRing()]+[atom.IsInRingSize(i) for i in list]

def bond_is_in_ring_list_one_hot(bond, allowable_set=None, encode_unknown=False):
          list = [3,4,5] # List denotes the size of cycles
          return [bond.IsInRing()]+[bond.IsInRingSize(i) for i in list]

# Almost the same as DMPNN; Not sure about chirlarity
class CanonicalAtomFeaturizer_without_AF_OH(fs.BaseAtomFeaturizer):
    def __init__(self, atom_data_field='h'):
        super().__init__(
            featurizer_funcs={atom_data_field: fs.ConcatFeaturizer(
                [
                 fs.atomic_number_one_hot,
                 fs.atom_degree_one_hot,
                 fs.atom_formal_charge_one_hot,
                 fs.atom_chiral_tag_one_hot,
                 fs.atom_chirality_type_one_hot, 
                 fs.atom_is_chiral_center,
                 fs.atom_total_num_H_one_hot,
                 fs.atom_hybridization_one_hot,
                 fs.atom_is_aromatic,
                 fs.atom_mass,
                 #fs.atom_explicit_valence_one_hot,
                 #fs.atom_implicit_valence_one_hot,                
                 #fs.atom_num_radical_electrons_one_hot,
                 #fs.atom_is_in_ring,
                 #atom_is_in_ring_list_one_hot
                 ]
            )})

# Same as DMPNN
class CanonicalBondFeaturizer(fs.BaseBondFeaturizer):
       def __init__(self, bond_data_field='e', self_loop=False):
        super(CanonicalBondFeaturizer, self).__init__(
            featurizer_funcs={bond_data_field: fs.ConcatFeaturizer(
                [fs.bond_type_one_hot,
                 fs.bond_is_conjugated,
                 fs.bond_is_in_ring,
                 fs.bond_stereo_one_hot,
                 #fs.bond_direction_one_hot,
                 #bond_is_in_ring_list_one_hot
                 ]
            )}, self_loop=self_loop)
        
BondFeaturizer = CanonicalBondFeaturizer()
AtomFeaturizer = CanonicalAtomFeaturizer_without_AF_OH()

In [ ]:
DATA = 'Tox21 - Random Scaffold Split - Seed 213'
# path to datasets
data = pd.read_csv('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz',compression='gzip').fillna(-1) # replacing NaN values with -1 to be able to create the mask
train = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/train_smiles', allow_pickle=True)
val = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/val_smiles', allow_pickle=True)
test = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/test_smiles', allow_pickle=True)
global_features = pd.read_csv('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/tox21_global_cdf_rdkit.zip',compression='zip')        
global_features = torch.Tensor(global_features.fillna(global_features.mean()).iloc[:,1:].values) 

In [ ]:
# Building a data generator for each splitted set

class Dataset_generator(torch.utils.data.Dataset): # for Tox21

    def __init__(self, splitted_data, data, gf):

      # In data NaN values are replaced with -1 #

      self.smiles = []
      self.dgl_graphs = []
      self.global_fetures = torch.empty(0)
      self.labels = torch.empty(0)
      self.masks = torch.empty(0)

      self.data_nan_0 = data.where(data != -1, 0) # replacing -1 with 0 for labels, however, it's only neccenary if mask is not used!

      # replacing non NaN values with 1 and NaNs with 0 for mask
      self.data_mask = data.where(data == -1, 1)
      self.data_mask = self.data_mask.where(self.data_mask != -1, 0)

      for i in range(len(splitted_data)):
         
        # finding its id in the main dataset and golbal feature set 
        id = pd.Index(data.smiles).get_loc(splitted_data[i])

        mol =  Chem.MolFromSmiles(splitted_data[i])

        self.smiles.append(splitted_data[i])

        self.dgl_graphs.append(dgllife.utils.mol_to_bigraph(mol, canonical_atom_order=False))

        self.dgl_graphs[i].ndata['h_0'] = AtomFeaturizer(mol)['h']

        # for the case encoutring a bondless compound; it can be deal while working on batches of graphs 
        try:
          self.dgl_graphs[i].edata['e_0'] = BondFeaturizer(mol)['e']
        except:
          print(splitted_data[i], "is bondless; I pass")

        self.global_fetures = torch.cat((self.global_fetures, gf[id].view(1,-1)),0)

        self.labels = torch.cat((self.labels, torch.Tensor(self.data_nan_0.iloc[id,:12]).view(1,-1)),0)
        
        self.masks = torch.cat((self.masks, torch.Tensor(self.data_mask.iloc[id,:12]).view(1,-1)),0)

    def __len__(self):

        return len(self.smiles)

    def __getitem__(self, idx):

        return  self.smiles[idx], self.dgl_graphs[idx], self.global_fetures[idx], self.labels[idx], self.masks[idx]

train_set = Dataset_generator(train, data, global_features)
val_set = Dataset_generator(val, data, global_features)
test_set = Dataset_generator(test, data, global_features)


def collate(batch):
    # batch is a list of triples (graphs, labels, masks)
    # Concatenate a sequence of graphs
    
    smiles = [e[0] for e in batch]
    
    graphs = [e[1] for e in batch]
    g = dgl.batch(graphs)

    gf = [e[2] for e in batch]
    gf = torch.stack(gf, 0)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[3] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[4] for e in batch]
    masks = torch.stack(masks, 0)

    return smiles, g, gf, labels, masks

def loader(batch_size=64):

    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False)
    
    return train_dataloader, val_dataloader, test_dataloader


RDKit WARNING: [16:09:34] WARNING: not removing hydrogen atom without neighbors


[I-].[K+] is bondless; I pass
[Hg+2] is bondless; I pass
[Ba+2] is bondless; I pass
[TlH2+] is bondless; I pass
[Cr+3] is bondless; I pass
[Fe+2] is bondless; I pass
[Co+2] is bondless; I pass
[PbH2+2] is bondless; I pass
[Fe+3] is bondless; I pass
[Cu+2] is bondless; I pass
[Cd+2] is bondless; I pass
[SnH2+2] is bondless; I pass
[Mn+2] is bondless; I pass
[Be+2] is bondless; I pass
[Zn+2] is bondless; I pass
[Br-].[Na+] is bondless; I pass
[Ca+2].[Cl-].[Cl-] is bondless; I pass
[SbH6+3] is bondless; I pass
[Ni+2] is bondless; I pass


# ***Scratch***

It's not important

In [ ]:
train_set = Dataset_generator(train, data, global_features)

[I-].[K+] is bondless; I pass
[Hg+2] is bondless; I pass
[Ba+2] is bondless; I pass
[TlH2+] is bondless; I pass
[Cr+3] is bondless; I pass
[Fe+2] is bondless; I pass
[Co+2] is bondless; I pass
[PbH2+2] is bondless; I pass
[Fe+3] is bondless; I pass
[Cu+2] is bondless; I pass
[Cd+2] is bondless; I pass
[SnH2+2] is bondless; I pass
[Mn+2] is bondless; I pass
[Be+2] is bondless; I pass
[Zn+2] is bondless; I pass
[Br-].[Na+] is bondless; I pass
[Ca+2].[Cl-].[Cl-] is bondless; I pass
[SbH6+3] is bondless; I pass
[Ni+2] is bondless; I pass


In [ ]:
# Testing
n = 6118
id = pd.Index(data.smiles).get_loc(train_set[n][0])
print(train_set[n])
print(data.iloc[id,:])
print(data_nan.iloc[id,:])
global_features[id]


('COc1nn(CSP(=S)(OC)OC)c(=O)s1', Graph(num_nodes=16, num_edges=32,
      ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
      edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([9.5474e-01, 6.5926e-02, 5.6989e-02, 4.2638e-02, 1.6386e-01, 4.1045e-02,
        1.5790e-02, 4.1783e-01, 8.5901e-03, 8.0434e-01, 1.2248e-02, 8.8020e-01,
        7.5892e-03, 5.8290e-01, 5.8904e-01, 2.6487e-01, 6.8936e-07, 3.7932e-01,
        1.2194e-01, 2.9572e-01, 9.5201e-01, 5.8561e-17, 1.0887e-06, 1.6207e-01,
        9.0172e-01, 1.7752e-01, 9.7443e-01, 9.1890e-01, 5.7003e-01, 9.1756e-01,
        9.8943e-01, 4.3509e-02, 1.9853e-01, 1.0000e+00, 1.6076e-01, 2.0851e-01,
        4.5532e-01, 7.1045e-02, 1.3114e-01, 6.5044e-01, 1.3114e-01, 8.3784e-01,
        7.4498e-01, 8.4419e-01, 7.0361e-02, 3.4010e-01, 1.2914e-01, 7.7518e-02,
        1.7358e-01, 6.1316e-02, 5.0000e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
        3.4723e-02, 8.6504e-01, 8.3758e-02, 9.5692e-01, 5.5287e-02, 8.

tensor([9.5474e-01, 6.5926e-02, 5.6989e-02, 4.2638e-02, 1.6386e-01, 4.1045e-02,
        1.5790e-02, 4.1783e-01, 8.5901e-03, 8.0434e-01, 1.2248e-02, 8.8020e-01,
        7.5892e-03, 5.8290e-01, 5.8904e-01, 2.6487e-01, 6.8936e-07, 3.7932e-01,
        1.2194e-01, 2.9572e-01, 9.5201e-01, 5.8561e-17, 1.0887e-06, 1.6207e-01,
        9.0172e-01, 1.7752e-01, 9.7443e-01, 9.1890e-01, 5.7003e-01, 9.1756e-01,
        9.8943e-01, 4.3509e-02, 1.9853e-01, 1.0000e+00, 1.6076e-01, 2.0851e-01,
        4.5532e-01, 7.1045e-02, 1.3114e-01, 6.5044e-01, 1.3114e-01, 8.3784e-01,
        7.4498e-01, 8.4419e-01, 7.0361e-02, 3.4010e-01, 1.2914e-01, 7.7518e-02,
        1.7358e-01, 6.1316e-02, 5.0000e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
        3.4723e-02, 8.6504e-01, 8.3758e-02, 9.5692e-01, 5.5287e-02, 8.0827e-01,
        5.6952e-08, 4.6631e-01, 0.0000e+00, 1.1424e-21, 2.4025e-23, 1.0269e-01,
        5.4838e-01, 4.7325e-01, 3.6088e-15, 8.3721e-01, 9.7265e-01, 8.6909e-01,
        3.1226e-01, 6.2848e-01, 4.6098e-

In [ ]:
train_dataloader, val_dataloader, test_dataloader = loader(1)

In [ ]:
for i, (_, mol_dgl_graph, fg, labels, masks) in enumerate(train_dataloader):
  break
mol_dgl_graph 

Graph(num_nodes=18, num_edges=36,
      ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
      edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)})

In [ ]:
train_set[5]

('Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl', Graph(num_nodes=12, num_edges=24,
       ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
       edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([9.9111e-01, 1.2006e-02, 1.3444e-02, 2.4111e-03, 3.3450e-02, 7.5539e-03,
         2.1564e-03, 1.7987e-02, 3.3799e-03, 3.4440e-02, 4.8310e-03, 1.3071e-01,
         3.8534e-03, 5.3349e-02, 4.0749e-10, 2.8232e-01, 6.8936e-07, 9.1771e-01,
         1.9674e-02, 2.5099e-11, 9.2584e-11, 5.8561e-17, 1.0887e-06, 2.3915e-11,
         9.9480e-01, 8.7876e-02, 9.9602e-03, 4.6250e-03, 3.8716e-03, 2.1555e-14,
         9.9431e-01, 9.1700e-03, 1.2489e-01, 1.0000e+00, 3.3636e-02, 2.0912e-02,
         2.0390e-01, 3.6221e-02, 4.1258e-02, 9.3943e-01, 4.1258e-02, 8.8595e-02,
         6.5742e-02, 7.3392e-02, 5.3147e-01, 3.7913e-02, 7.9191e-01, 1.9732e-02,
         8.9505e-02, 4.3608e-01, 5.1530e-02, 9.9386e-22, 1.4249e-01, 6.2963e-02,
         3.5821e-01, 4.8299e-15, 8.3758e-02, 4.0600e-02, 4.74

In [ ]:
train_set[0][-1]*train_set[0][-2]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# Testing
n = 118
id = pd.Index(data.smiles).get_loc(val_set[n][0])
print(val_set[n])
print(data.iloc[id,:])
print(data_nan.iloc[id,:])
global_features[id]


('O=C(Nc1ccccc1)c1cc([N+](=O)[O-])ccc1Cl', Graph(num_nodes=19, num_edges=40,
      ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
      edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([8.7940e-01, 1.8567e-01, 1.1331e-01, 6.5566e-02, 7.0570e-02, 1.0489e-01,
        6.3394e-02, 5.7267e-02, 5.6233e-02, 5.1236e-02, 6.1731e-02, 5.7541e-02,
        6.0515e-02, 5.3588e-02, 7.5999e-01, 7.1093e-01, 6.8936e-07, 5.3342e-01,
        1.3522e-01, 1.6094e-01, 7.7290e-01, 7.5902e-01, 4.6477e-01, 1.6919e-01,
        6.5065e-01, 1.1202e-01, 6.3283e-01, 5.1526e-01, 4.3470e-01, 2.1555e-14,
        6.3900e-01, 1.0630e-01, 1.3051e-01, 1.0000e+00, 1.0744e-01, 1.3359e-01,
        3.9963e-01, 1.0365e-01, 2.2107e-01, 1.2410e-01, 2.2107e-01, 5.7618e-01,
        3.9772e-01, 5.5605e-01, 3.8368e-01, 8.6674e-01, 5.2444e-01, 1.0113e-01,
        1.0985e-01, 4.3608e-01, 3.0517e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
        6.3529e-01, 4.8299e-15, 3.8583e-01, 1.6844e-01, 4.74

tensor([8.7940e-01, 1.8567e-01, 1.1331e-01, 6.5566e-02, 7.0570e-02, 1.0489e-01,
        6.3394e-02, 5.7267e-02, 5.6233e-02, 5.1236e-02, 6.1731e-02, 5.7541e-02,
        6.0515e-02, 5.3588e-02, 7.5999e-01, 7.1093e-01, 6.8936e-07, 5.3342e-01,
        1.3522e-01, 1.6094e-01, 7.7290e-01, 7.5902e-01, 4.6477e-01, 1.6919e-01,
        6.5065e-01, 1.1202e-01, 6.3283e-01, 5.1526e-01, 4.3470e-01, 2.1555e-14,
        6.3900e-01, 1.0630e-01, 1.3051e-01, 1.0000e+00, 1.0744e-01, 1.3359e-01,
        3.9963e-01, 1.0365e-01, 2.2107e-01, 1.2410e-01, 2.2107e-01, 5.7618e-01,
        3.9772e-01, 5.5605e-01, 3.8368e-01, 8.6674e-01, 5.2444e-01, 1.0113e-01,
        1.0985e-01, 4.3608e-01, 3.0517e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
        6.3529e-01, 4.8299e-15, 3.8583e-01, 1.6844e-01, 4.7447e-01, 3.4646e-01,
        5.6952e-08, 1.1745e-01, 0.0000e+00, 1.1424e-21, 2.4025e-23, 1.1646e-01,
        1.3928e-01, 5.7614e-21, 3.6088e-15, 1.4598e-01, 9.9374e-01, 1.1809e-10,
        8.3478e-01, 9.0550e-08, 4.6098e-

In [ ]:
# Testing
n = 188
id = pd.Index(data.smiles).get_loc(test_set[n][0])
print(test_set[n])
print(data.iloc[id,:])
print(data_nan.iloc[id,:])
global_features[id]


('CO[C@H](C(=O)[C@@H](O)[C@@H](C)O)C1Cc2cc3cc(O[C@H]4C[C@@H](O[C@H]5C[C@@H](O)[C@H](O)[C@@H](C)O5)[C@@H](O)[C@@H](C)O4)c(C)c(O)c3c(O)c2C(=O)[C@H]1O[C@H]1C[C@@H](O[C@H]2C[C@@H](O[C@H]3C[C@](C)(O)[C@H](O)[C@@H](C)O3)[C@H](O)[C@@H](C)O2)[C@H](O)[C@@H](C)O1', Graph(num_nodes=76, num_edges=166,
      ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
      edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([8.4747e-03, 9.9843e-01, 9.9794e-01, 9.9738e-01, 9.9744e-01, 9.9735e-01,
        9.9705e-01, 9.9703e-01, 9.9725e-01, 9.9625e-01, 9.9713e-01, 9.9600e-01,
        9.9584e-01, 9.9444e-01, 9.8448e-01, 9.9863e-01, 1.0000e+00, 9.9984e-01,
        1.9674e-02, 2.5099e-11, 9.7285e-01, 8.0085e-01, 1.0887e-06, 2.3915e-11,
        9.9112e-01, 9.9795e-01, 2.3443e-02, 2.1204e-02, 2.3266e-02, 9.7085e-01,
        2.8883e-01, 9.9765e-01, 9.9789e-01, 1.0000e+00, 9.9765e-01, 9.9373e-01,
        9.7394e-01, 9.9749e-01, 9.2809e-01, 9.5068e-01, 9.2809e-01, 1.8443e-01,
      

tensor([8.4747e-03, 9.9843e-01, 9.9794e-01, 9.9738e-01, 9.9744e-01, 9.9735e-01,
        9.9705e-01, 9.9703e-01, 9.9725e-01, 9.9625e-01, 9.9713e-01, 9.9600e-01,
        9.9584e-01, 9.9444e-01, 9.8448e-01, 9.9863e-01, 1.0000e+00, 9.9984e-01,
        1.9674e-02, 2.5099e-11, 9.7285e-01, 8.0085e-01, 1.0887e-06, 2.3915e-11,
        9.9112e-01, 9.9795e-01, 2.3443e-02, 2.1204e-02, 2.3266e-02, 9.7085e-01,
        2.8883e-01, 9.9765e-01, 9.9789e-01, 1.0000e+00, 9.9765e-01, 9.9373e-01,
        9.7394e-01, 9.9749e-01, 9.2809e-01, 9.5068e-01, 9.2809e-01, 1.8443e-01,
        2.3734e-01, 1.6914e-01, 8.7976e-02, 2.5847e-02, 2.8716e-02, 9.9567e-01,
        9.9830e-01, 9.9351e-01, 9.9981e-01, 9.9995e-01, 9.8276e-01, 9.8620e-01,
        6.3529e-01, 4.8299e-15, 3.8583e-01, 1.0000e+00, 9.9556e-01, 9.9927e-01,
        5.6952e-08, 9.5893e-01, 0.0000e+00, 9.9988e-01, 9.8405e-01, 9.9816e-01,
        9.9947e-01, 9.9979e-01, 9.9953e-01, 8.5084e-01, 1.7356e-22, 1.1809e-10,
        6.1801e-01, 9.0550e-08, 4.6098e-

In [ ]:
len(train)

6264

In [ ]:
len(val)

783

In [ ]:
len(test)

784

In [ ]:
val[12]

'CCCNCC(O)COc1ccccc1C(=O)CCc1ccccc1'

In [ ]:
test[12]

'CC(C)C1OCCN1CCOC(=O)NCCCCCCNC(=O)OCCN1CCOC1C(C)C'

In [ ]:
# Testing
n = 4118
id = pd.Index(data.smiles).get_loc(train_set[n][0])
print(train_set[n])
print(data.iloc[id,:])
print(data_nan.iloc[id,:])
global_features[id]


('C[C@]12CC[C@@H]3c4ccc(O)cc4C[C@@H](CCCCCCCCCS(=O)CCCC(F)(F)C(F)(F)F)[C@H]3[C@@H]1CC[C@@H]2O', Graph(num_nodes=41, num_edges=88,
      ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
      edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([2.2151e-02, 5.8517e-01, 9.2379e-01, 9.2662e-01, 9.3159e-01, 9.0275e-01,
        9.5166e-01, 9.6210e-01, 9.6513e-01, 9.7319e-01, 9.7860e-01, 9.8364e-01,
        9.8277e-01, 9.8689e-01, 9.2398e-01, 9.7446e-01, 6.8936e-07, 5.6763e-01,
        9.0848e-01, 9.9011e-01, 7.5664e-01, 5.8561e-17, 5.3771e-01, 4.0762e-01,
        7.5225e-13, 9.3492e-01, 1.7741e-01, 1.6526e-01, 2.2652e-01, 9.8293e-01,
        9.2123e-01, 9.1316e-01, 9.2752e-01, 1.0000e+00, 9.5025e-01, 9.2390e-01,
        8.2958e-01, 9.1807e-01, 5.9133e-01, 9.5674e-01, 5.9133e-01, 9.7613e-01,
        1.3581e-01, 9.8293e-01, 2.7025e-02, 1.9727e-02, 9.9502e-01, 9.0048e-01,
        9.3890e-01, 7.0728e-01, 1.2405e-01, 1.0000e+00, 1.4249e-01, 9.6456e-01,
       

tensor([2.2151e-02, 5.8517e-01, 9.2379e-01, 9.2662e-01, 9.3159e-01, 9.0275e-01,
        9.5166e-01, 9.6210e-01, 9.6513e-01, 9.7319e-01, 9.7860e-01, 9.8364e-01,
        9.8277e-01, 9.8689e-01, 9.2398e-01, 9.7446e-01, 6.8936e-07, 5.6763e-01,
        9.0848e-01, 9.9011e-01, 7.5664e-01, 5.8561e-17, 5.3771e-01, 4.0762e-01,
        7.5225e-13, 9.3492e-01, 1.7741e-01, 1.6526e-01, 2.2652e-01, 9.8293e-01,
        9.2123e-01, 9.1316e-01, 9.2752e-01, 1.0000e+00, 9.5025e-01, 9.2390e-01,
        8.2958e-01, 9.1807e-01, 5.9133e-01, 9.5674e-01, 5.9133e-01, 9.7613e-01,
        1.3581e-01, 9.8293e-01, 2.7025e-02, 1.9727e-02, 9.9502e-01, 9.0048e-01,
        9.3890e-01, 7.0728e-01, 1.2405e-01, 1.0000e+00, 1.4249e-01, 9.6456e-01,
        3.5821e-01, 4.8299e-15, 8.3758e-02, 1.6844e-01, 7.3166e-01, 7.2707e-01,
        5.6952e-08, 9.4696e-01, 9.3381e-01, 1.1424e-21, 9.4279e-01, 9.6414e-01,
        3.6613e-01, 4.6640e-01, 3.6088e-15, 1.4598e-01, 1.7356e-22, 9.6188e-01,
        2.7349e-01, 9.0550e-08, 9.9594e-

In [ ]:
data

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
train_set.data_mask

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,1
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,1
7827,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1,1
7828,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
7829,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1


In [ ]:
train_set.data_nan_0

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
data_nan

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
train_set[0]

('CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C', Graph(num_nodes=20, num_edges=40,
       ndata_schemes={'h_0': Scheme(shape=(135,), dtype=torch.float32)}
       edata_schemes={'e_0': Scheme(shape=(12,), dtype=torch.float32)}), tensor([9.6758e-01, 5.9713e-02, 1.7813e-01, 3.0837e-01, 2.5721e-01, 1.3395e-01,
         2.9016e-01, 2.2136e-01, 2.3168e-01, 1.5390e-01, 2.7137e-01, 1.8217e-01,
         3.0554e-01, 2.0713e-01, 4.0749e-10, 2.6487e-01, 6.8936e-07, 4.2420e-01,
         1.9674e-02, 8.4760e-01, 9.2584e-11, 5.8561e-17, 6.5958e-01, 7.7466e-01,
         7.5225e-13, 1.1243e-01, 7.1569e-01, 4.4008e-01, 1.9227e-01, 8.4676e-01,
         9.0475e-01, 1.3706e-01, 8.7870e-02, 1.0000e+00, 3.2919e-01, 4.9647e-01,
         5.1656e-01, 1.5603e-01, 4.1565e-01, 1.2659e-01, 4.1565e-01, 3.4639e-01,
         2.7092e-01, 3.3832e-01, 7.2101e-01, 8.6413e-01, 5.9004e-01, 2.3054e-01,
         1.0932e-01, 4.3608e-01, 1.2405e-01, 9.9386e-22, 1.4249e-01, 6.2963e-02,
         3.5821e-01, 4.8299e-15, 8.3758e-02, 8.4739e-02, 

In [ ]:
data.smiles[id]

'CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C'

In [ ]:
train.shape

(6264,)

In [ ]:
len(train)

6264

In [ ]:
train[1]

'O=S(=O)(Cl)c1ccccc1'

In [ ]:
mol = Chem.MolFromSmiles(train[1])

In [ ]:
BondFeaturizer(mol)['e']

tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0

In [ ]:
global_features[1].view(1,-1).shape

torch.Size([1, 200])

In [ ]:
a = torch.Tensor(data.iloc[1,:12]).view(1,-1)
a.dtype

torch.float32

In [ ]:
a

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
torch.Tensor(data.iloc[1,:12].astype(float).values).view(1,-1)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
data.iloc[1,:12].astype(float).values

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
torch.Tensor(data.iloc[1,:12].values).view(1,-1)

TypeError: ignored

In [ ]:
data = pd.read_csv('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz',compression='gzip').fillna(0)

In [ ]:
r = data.iloc[:,:12].values

In [ ]:
type(r)

numpy.ndarray

In [ ]:
len(data)

7831

In [ ]:
r[1,1] == 0.0

True

In [ ]:
data.iloc[0,:]

NR-AR                                       0
NR-AR-LBD                                   0
NR-AhR                                      1
NR-Aromatase                                0
NR-ER                                       0
NR-ER-LBD                                   0
NR-PPAR-gamma                               0
SR-ARE                                      1
SR-ATAD5                                    0
SR-HSE                                      0
SR-MMP                                      0
SR-p53                                      0
mol_id                                TOX3021
smiles           CCOc1ccc2nc(S(N)(=O)=O)sc2c1
Name: 0, dtype: object

In [ ]:
r[0]

array([0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [ ]:
for i in range(len(data)):
  if r[i,4] != 0:
    print(i)
    break

7


In [ ]:
b

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [ ]:
b = data.replace({0: 1}).iloc[:,:12].values

In [ ]:
r[7]

array([0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.])

In [ ]:
for i in range(len(data)):
  if b[i,4] != 1:
    print(i)
    break

In [ ]:
data

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
data_nan = pd.read_csv('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz',compression='gzip')

In [ ]:
data_nan

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
a=data_nan.iloc[0,3] 
a.size

1

In [ ]:
type(a)

numpy.float64

In [ ]:
data_nan.where(data == np.nan, 44)

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
1,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
2,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
3,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
4,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
7827,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
7828,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44
7829,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44,44


In [ ]:
data_nan.mask(data == pd.NA, 44)

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
data_nan.iloc[0,3].values

In [ ]:
data_nan.mask(data == np.nan, 44)

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
data_na_neg = pd.read_csv('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz',compression='gzip').fillna(-1)
data_na_neg

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
data_na_neg_masked = data_na_neg.where(data_na_neg == -1, 1)
data_na_neg_masked.where(data_na_neg_masked != -1, 0)

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,1
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,1
7827,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1,1
7828,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
7829,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1


# ***Scratch***

In [ ]:
data_na_neg = pd.read_csv('https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz',compression='gzip').fillna(-1)
data_na_neg

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
a = data_na_neg.where(data_na_neg != -1, 0)
a

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [ ]:
train[0]

'CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C'

In [ ]:
id = data.smiles.index.get_indexer_for(train[0])
(train[0])
id

TypeError: ignored

In [ ]:
fid = 5000

In [ ]:
id = pd.Index(data.smiles).get_loc(train[fid])
id

1695

In [ ]:
train[fid]

'C[C@]12CC[C@@H]3[C@H]4CCC(=O)C=C4CC[C@H]3[C@@H]1CC[C@@H]2OC(=O)CCc1ccccc1'

In [ ]:
data.smiles[id]

'C[C@]12CC[C@@H]3[C@H]4CCC(=O)C=C4CC[C@H]3[C@@H]1CC[C@@H]2OC(=O)CCc1ccccc1'

In [ ]:
train = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/train_smiles', allow_pickle=True)
# valid = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/val_smiles')
# test = np.load('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/test_smiles')

In [ ]:
train[1]

'O=S(=O)(Cl)c1ccccc1'

In [ ]:
global_features = pd.read_csv('/content/drive/MyDrive/GMLG_Research/Data/DeepChem_RandomScaffoldSplit/Tox21_seed_213/tox21_global_cdf_rdkit.zip',compression='zip')

In [ ]:
global_features

,"('RDKit2DNormalized_calculated', <class 'bool'>)","('BalabanJ', <class 'numpy.float64'>)","('BertzCT', <class 'numpy.float64'>)","('Chi0', <class 'numpy.float64'>)","('Chi0n', <class 'numpy.float64'>)","('Chi0v', <class 'numpy.float64'>)","('Chi1', <class 'numpy.float64'>)","('Chi1n', <class 'numpy.float64'>)","('Chi1v', <class 'numpy.float64'>)","('Chi2n', <class 'numpy.float64'>)","('Chi2v', <class 'numpy.float64'>)","('Chi3n', <class 'numpy.float64'>)","('Chi3v', <class 'numpy.float64'>)","('Chi4n', <class 'numpy.float64'>)","('Chi4v', <class 'numpy.float64'>)","('EState_VSA1', <class 'numpy.float64'>)","('EState_VSA10', <class 'numpy.float64'>)","('EState_VSA11', <class 'numpy.float64'>)","('EState_VSA2', <class 'numpy.float64'>)","('EState_VSA3', <class 'numpy.float64'>)","('EState_VSA4', <class 'numpy.float64'>)","('EState_VSA5', <class 'numpy.float64'>)","('EState_VSA6', <class 'numpy.float64'>)","('EState_VSA7', <class 'numpy.float64'>)","('EState_VSA8', <class 'numpy.float64'>)","('EState_VSA9', <class 'numpy.float64'>)","('ExactMolWt', <class 'numpy.float64'>)","('FpDensityMorgan1', <class 'numpy.float64'>)","('FpDensityMorgan2', <class 'numpy.float64'>)","('FpDensityMorgan3', <class 'numpy.float64'>)","('FractionCSP3', <class 'numpy.float64'>)","('HallKierAlpha', <class 'numpy.float64'>)","('HeavyAtomCount', <class 'numpy.float64'>)","('HeavyAtomMolWt', <class 'numpy.float64'>)","('Ipc', <class 'numpy.float64'>)","('Kappa1', <class 'numpy.float64'>)","('Kappa2', <class 'numpy.float64'>)","('Kappa3', <class 'numpy.float64'>)","('LabuteASA', <class 'numpy.float64'>)","('MaxAbsEStateIndex', <class 'numpy.float64'>)",...,"('fr_hdrzone', <class 'numpy.float64'>)","('fr_imidazole', <class 'numpy.float64'>)","('fr_imide', <class 'numpy.float64'>)","('fr_isocyan', <class 'numpy.float64'>)","('fr_isothiocyan', <class 'numpy.float64'>)","('fr_ketone', <class 'numpy.float64'>)","('fr_ketone_Topliss', <class 'numpy.float64'>)","('fr_lactam', <class 'numpy.float64'>)","('fr_lactone', <class 'numpy.float64'>)","('fr_methoxy', <class 'numpy.float64'>)","('fr_morpholine', <class 'numpy.float64'>)","('fr_nitrile', <class 'numpy.float64'>)","('fr_nitro', <class 'numpy.float64'>)","('fr_nitro_arom', <class 'numpy.float64'>)","('fr_nitro_arom_nonortho', <class 'numpy.float64'>)","('fr_nitroso', <class 'numpy.float64'>)","('fr_oxazole', <class 'numpy.float64'>)","('fr_oxime', <class 'numpy.float64'>)","('fr_para_hydroxylation', <class 'numpy.float64'>)","('fr_phenol', <class 'numpy.float64'>)","('fr_phenol_noOrthoHbond', <class 'numpy.float64'>)","('fr_phos_acid', <class 'numpy.float64'>)","('fr_phos_ester', <class 'numpy.float64'>)","('fr_piperdine', <class 'numpy.float64'>)","('fr_piperzine', <class 'numpy.float64'>)","('fr_priamide', <class 'numpy.float64'>)","('fr_prisulfonamd', <class 'numpy.float64'>)","('fr_pyridine', <class 'numpy.float64'>)","('fr_quatN', <class 'numpy.float64'>)","('fr_sulfide', <class 'numpy.float64'>)","('fr_sulfonamd', <class 'numpy.float64'>)","('fr_sulfone', <class 'numpy.float64'>)","('fr_term_acetylene', <class 'numpy.float64'>)","('fr_tetrazole', <class 'numpy.float64'>)","('fr_thiazole', <class 'numpy.float64'>)","('fr_thiocyan', <class 'numpy.float64'>)","('fr_thiophene', <class 'numpy.float64'>)","('fr_unbrch_alkane', <class 'numpy.float64'>)","('fr_urea', <class 'numpy.float64'>)","('qed', <class 'numpy.float64'>)"
0,True,0.934799,0.177418,0.046765,0.033326,0.060566,0.039962,0.029073,0.141434,0.024657,0.228561,0.025139,0.268049,0.034317,0.282035,7.426378e-01,4.562858e-01,6.893569e-07,0.179159,0.509277,4.086641e-01,9.258417e-11,6.674678e-01,0.477870,1.583304e-01,5.915087e-01,0.077751,0.974430,0.973245,0.940471,0.274299,0.925753,0.043509,0.087954,0.999999,0.049894,0.031270,0.281892,0.043682,0.104644,...,0.168149,0.165451,1.171103e-13,0.0,0.164669,0.166925,0.000000,5.100713e-08,7.105427e-15,0.154654,2.794209e-22,0.0,0.16764,6.314993e-25,0.168186,0.009089,0.168363,8.265423e-11,0.156346,0.000000,0.000

In [ ]:
global_features = torch.Tensor(global_features.fillna(global_features.mean()).iloc[:,1:].values)

In [ ]:
global_features.shape

torch.Size([7831, 200])